In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle
from scipy.integrate import quad ## FUNCTIONS TO IMPLEMENT GAUSS-QUADRATURE
# from scipy.integrate import quad_vec ## FUNCTIONS TO IMPLEMENT GAUSS-QUADRATURE
from scipy.special import hermite,legendre
from scipy.linalg import eigh
import time
np.random.seed(20)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [2]:
t1=time.time()

In [3]:
# This will set the default to float32 or float64 but not float16
torch.set_default_dtype(torch.float32)

In [4]:
# current_device = torch.cuda.current_device()
# torch.cuda.get_device_properties(current_device)

In [5]:
# Set the default tensor type to use GPU if available
if torch.cuda.is_available():
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
else:
    torch.set_default_tensor_type(torch.FloatTensor)
torch.set_default_tensor_type(torch.FloatTensor)

C:\Users\CMMRG\anaconda3\Lib\site-packages\torch\__init__.py:749: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ..\torch\csrc\tensor\python_tensor.cpp:433.)
  _C._set_default_tensor_type(t)


In [6]:
# Define the orthonormal probabilistic Hermite polynomial
def orthonormal_prob_hermite(n):
    return lambda x: hermite(n)(x/np.sqrt(2.0)) / np.sqrt((2.0)**n * np.math.factorial(n))

# Define the orthonormal probabilistic Legendre polynomial
def orthonormal_legendre(n):
    return lambda x: legendre(n)(x) * np.sqrt(n+0.5)

# Define the integrand for scalar product
def scalar_int(i,j):
    return lambda x:orthonormal_prob_hermite(i)(x) * orthonormal_prob_hermite(j)(x) * np.exp(-(x**2/2))/np.sqrt(2*np.pi)

# Define the integrand for triple product
def triple_int(i,j,k):
    return lambda x:orthonormal_prob_hermite(i)(x) * orthonormal_prob_hermite(j)(x)* orthonormal_prob_hermite(k)(x) * np.exp(-(x**2/2))/np.sqrt(2*np.pi)

# Integrate to verify orthonormality
integral1, _ = quad(scalar_int(2,2), -np.inf, np.inf)
integral2, _ = quad(scalar_int(2,3), -np.inf, np.inf)
print(f"The scalar product of the orthonormal probabilistic Hermite polynomials H_2(x) and H_2(x) is approximately {integral1}")
print(f"The scalar product of the orthonormal probabilistic Hermite polynomials H_2(x) and H_3(x) is approximately {integral2}")

The scalar product of the orthonormal probabilistic Hermite polynomials H_2(x) and H_2(x) is approximately 1.0000000000000022
The scalar product of the orthonormal probabilistic Hermite polynomials H_2(x) and H_3(x) is approximately 0.0


C:\Users\CMMRG\AppData\Local\Temp\ipykernel_19452\293942335.py:3: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  return lambda x: hermite(n)(x/np.sqrt(2.0)) / np.sqrt((2.0)**n * np.math.factorial(n))


In [7]:
k=1
con = 1
particles_per_unit=50*k
dx = con/particles_per_unit
ratio=1.6
h=ratio*dx
c_ = 2 * h
q=1
max_ord=15
n_samples=10000

J = particles_per_unit**2 #total number of particles
rho = 1000*torch.ones(J)  ## for steel
mass = rho * dx**2
rho0=rho
vis=0.05
T = 0.3               # Total time of integration
dt = 0.003          # Time step
N = int(T/dt)
N  

100

In [8]:
def find_combinations(q, max_ord,current_sum=0, current_combination=None, all_combinations=None):
    if current_combination is None:
        current_combination = []
    if all_combinations is None:
        all_combinations = []
        
    if len(current_combination) == q:
        if current_sum < max_ord:
            all_combinations.append(current_combination[:])
        return all_combinations
    
    for i in range(max_ord - current_sum):
        current_combination.append(i)
        find_combinations(q, max_ord,current_sum + i, current_combination, all_combinations)
        current_combination.pop()

    return all_combinations

In [9]:
index=np.array(find_combinations(q,max_ord))
P=index.shape[0]
P

15

In [10]:
index

array([[ 0],
       [ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 5],
       [ 6],
       [ 7],
       [ 8],
       [ 9],
       [10],
       [11],
       [12],
       [13],
       [14]])

In [11]:
x_values = torch.linspace(0, 1*con, particles_per_unit)
y_values = torch.linspace(0, 1*con, particles_per_unit)
# Create a meshgrid
X, Y = torch.meshgrid(x_values, y_values)
# print(X.shape)

# Flatten the meshgrid arrays for 1D function application
X_flat = X.flatten()
Y_flat = Y.flatten()

temp=tuple(zip(X_flat,Y_flat))
temp=torch.tensor(temp)
coords=torch.zeros(P,J,2)
coords[0]=temp

C:\Users\CMMRG\anaconda3\Lib\site-packages\torch\functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3588.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [12]:
values=torch.zeros(size=(2,P,J))
# values[0,0,:]=0.25*torch.sin(2 * np.pi * X_flat) * torch.sin(2 * np.pi * Y_flat);
# values[1,0,:]=-0.1*torch.sin(2 * np.pi * X_flat) * torch.sin(2 * np.pi * Y_flat);
for i in range(P):
    print(i)
    val=torch.zeros_like(u0_mean)
    for j in range(q):
        k = index[i][j]
        temp,_=quad(lambda x: orthonormal_prob_hermite(k)(x) * (0.05*x+0.25) * np.exp(-(x**2/2))/np.sqrt(2*np.pi), -np.inf, np.inf)
        # temp2,_=quad(lambda x: orthonormal_prob_hermite(k)(x) * -(0.01*x+0.1) * np.exp(-(x**2/2))/np.sqrt(2*np.pi), -np.inf, np.inf)
            # print(f"{k} for nananaaaa naaa{temp}")
        val[0]+=temp* torch.sin(2 * np.pi * X_flat) * torch.sin(2 * np.pi * Y_flat);
        val[1]+=temp* torch.sin(2 * np.pi * X_flat) * torch.sin(2 * np.pi * Y_flat);
            
    values[:,i,:]=val

0


NameError: name 'u0_mean' is not defined

In [ ]:
ut = torch.zeros((2, P, J, N+1))
ut[:,:,:,0]=torch.tensor(values)
values

In [ ]:
pre_b=torch.zeros((max_ord,max_ord),device='cpu')
for i in range(max_ord):
    for j in range(max_ord):
        pre_b[i,j],_=quad(lambda x: orthonormal_prob_hermite(i)(x)*orthonormal_prob_hermite(j)(x)* np.exp(-(x**2/2))/np.sqrt(2*np.pi), -np.inf, np.inf)
        if(abs(pre_b[i,j])<1e-10):
            pre_b[i,j]=0
pre_b

In [ ]:
pre_c=torch.zeros((max_ord,max_ord,max_ord),device='cpu')
for i in range(max_ord):
    for j in range(max_ord):
        for k in range(max_ord):
            pre_c[i,j,k],_=quad(lambda x: orthonormal_prob_hermite(i)(x)*orthonormal_prob_hermite(j)(x)* orthonormal_prob_hermite(k)(x) *np.exp(-(x**2/2))/np.sqrt(2*np.pi), -np.inf, np.inf)
            if(abs(pre_c[i,j,k])<1e-10):
                pre_c[i,j,k]=0
pre_c

In [ ]:
def cal_b(i, pre_b, index):
    print(f"Starting {i}th iteration")
    b = torch.zeros(size=(P,),device='cpu')
    for j in range(P):
        b[j]=1
        for k in range(q):
            b[j]*=pre_b[index[i][k],index[j][k]]
    return b

In [ ]:
index=torch.tensor(index,device='cpu')

In [ ]:
def cal_c(i, pre_c, index):
    print(f"Starting {i}th iteration")
    P, q = index.shape
#     index=torch.tensor(index)
    # Initialize the output tensor c with ones
    c = torch.ones((P, P), dtype=pre_c.dtype, device=pre_c.device)
    
    # Iterate over the range Q and perform element-wise multiplication for all combinations
    for l in range(q):
        idx_i = index[i, l]
        idx_j = index[:, l].unsqueeze(1).expand(P, P)
        idx_k = index[:, l].unsqueeze(0).expand(P, P)
        c *= pre_c[idx_i, idx_j, idx_k]
    
    return c

In [ ]:
import time
import multiprocessing
from joblib import Parallel, delayed
import sys


In [ ]:
num_cores=multiprocessing.cpu_count()
num_cores

In [ ]:

start = time.time()

b = Parallel(n_jobs=num_cores)(delayed(cal_b)(i, pre_b, index) for i in range(P))
c = Parallel(n_jobs=num_cores)(delayed(cal_c)(i, pre_c, index) for i in range(P))

end = time.time()

In [ ]:
print(f"Total time taken {(end-start)/60} mins")

In [ ]:
b=torch.stack(b,dim=0)
b.shape

In [ ]:
c=torch.stack(c,dim=0)
c.shape

In [ ]:
def compute_nlist(coords,side_length=float(con)):
    # Compute pairwise distance
    # Shifts
    eps = 0.1*h
    shifts = [
        (0, 0)        # Original coordinates
        # (side_length, 0),  # Shift right
        # (-side_length, 0), # Shift left
        # (0, side_length),  # Shift up
        # (0, -side_length), # Shift down 
        # (side_length + eps, -side_length - eps),  # Shift right - down
        # (-side_length - eps, side_length + eps), # Shift left - up
        # (side_length + eps, side_length + eps),  # Shift right - up
        # (-side_length - eps, -side_length - eps)  # Shift left - down 
    ]
    n_particles=coords.size()[0]
    # Expand the coordinates
    
    all_coords = []
    for shift in shifts:
        shifted_coords = coords + torch.tensor(shift,dtype=coords.dtype)
        all_coords.append(shifted_coords)

    
    all_coords = torch.cat(all_coords, dim=0)
    
    # Compute pairwise distances
    dX = all_coords[:, 0][:, None] - coords[:, 0][None, :]
    dY = all_coords[:, 1][:, None] - coords[:, 1][None, :]
    # print(dX.shape)
    # half_domain = side_length / 2
    # dX = (dX + half_domain) % side_length - half_domain
    # dY = (dY + half_domain) % side_length - half_domain
    
    distances = torch.sqrt(dX**2 + dY**2)

    # Create the neighbor list using a mask for distances < c and excluding self-distances
    neighbor_mask = (distances <= c_) & (distances > 0)
    # print(neighbor_mask.shape)
    # Compute the neighbor list
    n_list = [[torch.nonzero(neighbor_mask[i]).squeeze() % n_particles,(dX[i][neighbor_mask[i]]),(dY[i][neighbor_mask[i]])] for i in range(J)]
    
    return n_list

n_list=compute_nlist(coords[0])
len(n_list[0][1])

In [ ]:
n_list[0][0]

In [ ]:
def d_CubicSpline(x, y, h, domain_size=1):
    alpha = 15/(7*np.pi*h**2)


    r = torch.sqrt(x**2 + y**2)
    q = r / h
    mask1 = q <= 1
    mask2 = (q > 1) & (q <= 2)
    dz_dx = torch.zeros_like(q)
    dz_dy = torch.zeros_like(q)

    dz_dx[mask1] = alpha * (-2*q[mask1] + 3/2*q[mask1]**2) * x[mask1] / (r[mask1] * h)
    dz_dy[mask1] = alpha * (-2*q[mask1] + 3/2*q[mask1]**2) * y[mask1] / (r[mask1] * h)
    dz_dx[mask2] = alpha * (-1/2*(2-q[mask2])**2) * x[mask2] / (r[mask2] * h)
    dz_dy[mask2] = alpha * (-1/2*(2-q[mask2])**2) * y[mask2] / (r[mask2] * h)
    
    return dz_dx, dz_dy


In [ ]:
def compute_A(x, y, h, m_j, rho_j, epsilon=1e-8):
    alpha = 15 / (7 * np.pi * h ** 2)
    r = torch.sqrt(x**2 + y**2)
    q = r / h
    mask1 = q <= 1
    mask2 = (q > 1) & (q <= 2)
    
    W_xx = torch.zeros_like(q)
    W_yy = torch.zeros_like(q)
    
    W_xx[mask1] = alpha * (-2 * q[mask1] + 3/2 * q[mask1]**2) * (x[mask1]) / (r[mask1] * h)
    W_yy[mask1] = alpha * (-2 * q[mask1] + 3/2 * q[mask1]**2) * (y[mask1]) / (r[mask1] * h)
    W_xx[mask2] = alpha * (-1/2 * (2 - q[mask2])**2) * (x[mask2]) / (r[mask2] * h)
    W_yy[mask2] = alpha * (-1/2 * (2 - q[mask2])**2) * (y[mask2]) / (r[mask2] * h)
    
    A_xx = -torch.sum(m_j * x / rho_j * W_xx)
    A_xy = -torch.sum(m_j * x / rho_j * W_yy)
    A_yx = -torch.sum(m_j * y / rho_j * W_xx)
    A_yy = -torch.sum(m_j * y / rho_j * W_yy)
    
    A_inv = torch.linalg.inv(torch.tensor([[A_xx, A_xy], [A_yx, A_yy]]))
    return A_inv

def d_CubicSpline_corrected(x, y, h, m_j, rho_j):
    # Compute the gradient of the kernel
    dz_dx, dz_dy = d_CubicSpline(x, y, h)
    
    # Compute the correction matrix A_inv
    B = compute_A(x, y, h, m_j, rho_j)
    
    # Apply the gradient correction
    dz_dx_corrected = B[0, 0] * dz_dx + B[0, 1] * dz_dy
    dz_dy_corrected = B[1, 0] * dz_dx + B[1, 1] * dz_dy
    
    return dz_dx_corrected, dz_dy_corrected

In [ ]:
def SPH(c,h,comp,n_list=n_list):
    cd=torch.zeros(size=(2,P,c.shape[1]))
    for i,data in enumerate(n_list):
        neighbors = data[0]
        c_i = c[:,i][:,np.newaxis]  # Broadcasting to create the necessary shape
        c_i = torch.tensor(np.repeat(c_i.cpu(), len(neighbors), axis=1),device=cd.device)
        c_j = c[:,neighbors]
        rho_neighbors = rho[neighbors]
        mass_neighbors = mass[neighbors]
        
        c_diff = c_i - c_j
        # dx,dy=d_CubicSpline_corrected(data[1],data[2], h, mass_neighbors, rho_neighbors)
        dx,dy=d_CubicSpline(data[1],data[2],h)
        cd[0,:,i] = torch.einsum("N,PN->P", mass_neighbors / rho_neighbors * dx, c_diff)
        cd[1,:,i] = torch.einsum("N,PN->P", mass_neighbors / rho_neighbors * dy, c_diff)
    return cd

In [ ]:
def SPH_d(c,h,n_list=n_list):
    cd=torch.zeros_like(c)
    for i,data in enumerate(n_list):
        neighbors = data[0]
        c_i = c[:,:,i][:,:,np.newaxis]  # Broadcasting to create the necessary shape
        c_i = torch.tensor(np.repeat(c_i.cpu(), len(neighbors), axis=2),device=cd.device)
        c_j = c[:,:,neighbors]
        rho_neighbors = rho[neighbors]
        mass_neighbors = mass[neighbors]
        c_diff = (c_i - c_j)
        # dx,dy=d_CubicSpline_corrected(data[1],data[2], h, mass_neighbors, rho_neighbors)
        dx,dy=d_CubicSpline(data[1],data[2],h)
        cd[0,:,i] = torch.einsum("N,PN->P",mass_neighbors / rho_neighbors * dx, c_diff[0,:,:])
        cd[1,:,i] = torch.einsum("N,PN->P",mass_neighbors / rho_neighbors * dy, c_diff[1,:,:])
    return cd

In [ ]:
coordinates=torch.zeros((P,J,2,N+1))
coordinates[:,:,:,0]=coords

In [ ]:
# coords[1,:51,0]

In [ ]:
x_mask = torch.logical_or(coords[0,:,0] == 0, coords[0,:,0] == 1)
y_mask = torch.logical_or(coords[0,:,1] == 0, coords[0,:,1] == 1)

In [ ]:
x_mask[:100]

In [ ]:
ut.shape

In [ ]:
for n in range(1, N+1):
    # Predictor step
    n_list = compute_nlist(coords[0])
    u = ut[:, :, :, n - 1]
    
    # Derivatives for predictor
    ux_d = SPH(u[0], h, 0)
    uy_d = SPH(u[1], h, 1)
    ux_dd = SPH_d(ux_d, h)
    uy_dd = SPH_d(uy_d, h)
    
    # Compute right-hand side for predictor
    rhsx = torch.einsum('ji,ki,jlk->li', u[0], ux_d[0], c) + torch.einsum('ji,ki,jlk->li', u[1], ux_d[1], c) \
           - vis * (ux_dd[0] + ux_dd[1])
    rhsy = torch.einsum('ji,ki,jlk->li', u[0], uy_d[0], c) + torch.einsum('ji,ki,jlk->li', u[1], uy_d[1], c) \
           - vis * (uy_dd[0] + uy_dd[1])

    ut[0, :, :, n] = u[0] - rhsx * dt
    ut[1, :, :, n] = u[1] - rhsy * dt
    
    
    # Boundary conditions
    ut[0, :, torch.logical_or(x_mask,y_mask), n] = 0
    ut[1, :, torch.logical_or(x_mask,y_mask), n] = 0

    coords = coords +  (ut[:,:,:,n]).transpose(0,2).transpose(0,1) * dt
    coordinates[:,:,:,n]=coords
    print(f"Completed the {n}^th timestep")

In [ ]:
import pickle

with open('results_pce4_1_new.pkl', 'wb') as file:
    pickle.dump(ut.numpy(), file)
with open('coords_pce4_1_new.pkl', 'wb') as file:
    pickle.dump(coordinates.numpy(), file)

In [ ]:
t2=time.time()

In [ ]:
print(f"Total time taken {(t2-t1)/60} mins")